In [1]:
pip install pulp

Looking in indexes: https://artifactory-haproxy.service.intradsm1.consul.csnzoo.com:8099/artifactory/api/pypi/python/simple, https://artifactory-haproxy.service.intradsm1.consul.csnzoo.com:8099/artifactory/api/pypi/pypi-mirror/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pulp

In [3]:
# Define the problem variables
origins = ["origin1", "origin2"]
destinations = ["destination1", "destination2"]
carriers = ["carrier1", "carrier2"]
service_strings = ["string1", "string2"]
weeks = ["week1", "week2"]
costs = {(o, d, c): 100 for o in origins for d in destinations for c in carriers}  # replace with your actual cost data
transit_times = {(o, d, s): 5 for o in origins for d in destinations for s in service_strings}  # replace with your actual transit time data
capacities = {(o, d, c): 100 for o in origins for d in destinations for c in carriers}  # replace with your actual capacity data
shipments = [{"order_number": 1, "origin": "origin1", "destination": "destination1", "week": "week1"},
             {"order_number": 2, "origin": "origin2", "destination": "destination2", "week": "week2"}]

In [5]:
# Define the optimization model
model= pulp.LpProblem("Carrier Allocation Problem", pulp.LpMinimize)

In [6]:
# Define the decision variables
x = pulp.LpVariable.dicts("ShipmentAllocation", [(s["order_number"], o, d, c, s["week"]) for o in origins for d in destinations for c in carriers for s in shipments],
                          lowBound=0, upBound=1, cat=pulp.LpInteger)

In [18]:
# Define the objective function
model += pulp.lpSum([costs[(o, d, c)] * transit_times.get((o, d, s.get("week", None), s.get("service", "")), 0) * x[(s["order_number"], o, d, c, s.get("week", None))] for o in origins for d in destinations for c in carriers for s in shipments])

In [21]:
# Define the constraints
for o in origins:
    for d in destinations:
        for c in carriers:
            model += pulp.lpSum([x[(s["order_number"], o, d, c, s["week"])] for s in shipments if s["origin"] == o and s["destination"] == d]) <= capacities[(o, d, c)] * 0.9
for s in shipments:
    model += pulp.lpSum([x[(s["order_number"], o, d, c, s["week"])] for c in carriers]) == 1
    model += pulp.lpSum([transit_times[(o, d, s.get("week", None), s.get("service", ""))] * x[(s["order_number"], o, d, c, s["week"])] for c in carriers if (o, d, s.get("week", None), s.get("service", "")) in transit_times]) <= pulp.lpSum([transit_times[(o, d, s.get("week", None), s.get("service", ""))] * x[(s["order_number"], o, d, c, s["week"])] for c in carriers if (o, d, s.get("week", None), s.get("service", "")) in transit_times])
    model += pulp.lpSum([costs[(o, d, c)] * transit_times[(o, d, s.get("week", None), s.get("service", ""))] * x[(s["order_number"], o, d, c, s["week"])] for c in carriers if (o, d, s.get("week", None), s.get("service", "")) in transit_times]) <= pulp.lpSum([costs[(o, d, c)] * transit_times[(o, d, s.get("week", None), s.get("service", ""))] * x[(s["order_number"], o, d, c, s["week"])] for c in carriers if (o, d, s.get("week", None), s.get("service", "")) in transit_times])
    for c in carriers:
        model += pulp.lpSum([x[(s["order_number"], o, d, c, s["week"])] for s in shipments if s["origin"] == o and s["destination"] == d]) <= capacities[(o, d, c)]

In [22]:
# Solve the optimization model
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/af8022b4ffc24248bdcfd3e92f653822-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/af8022b4ffc24248bdcfd3e92f653822-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 41 COLUMNS
At line 79 RHS
At line 116 BOUNDS
At line 124 ENDATA
Problem MODEL has 36 rows, 7 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were

1

In [36]:
# Store the results in a separate list of pending shipments
pending_shipments = []
for s in shipments:
    for o in origins:
        for d in destinations:
            for c in carriers:
                for w in weeks:
                    if (s["order_number"], o, d, c, w) in x and x[(s["order_number"], o, d, c, w)].value() == 1:
                        pending_shipments.append({
                            "order_number": s["order_number"],
                            "origin": o,
                            "destination": d,
                            "carrier": c,
                            "service_string": "some_service_string",  # replace "some_service_string" with the actual service string selected by the optimization model
                            "week": w
                        })
                        # you may also want to add other information such as the cost and transit time of the allocation

In [37]:
def get_service_strings(origin, destination):
    """
    Returns a list of available service strings for a given origin and destination.
    """
    if origin == "origin1" and destination == "destination1":
        return ["service_string1", "service_string2", "service_string3"]
    elif origin == "origin2" and destination == "destination2":
        return ["service_string4", "service_string5"]
    else:
        return []

In [38]:
from collections import namedtuple

# define named tuple for x dictionary keys
XKey = namedtuple("XKey", ["order_number", "origin", "destination", "carrier", "week"])

# create the x dictionary with named tuple keys
x = {XKey(*k): v for k, v in x.items()}

# loop through the shipments and allocate carriers
pending_shipments = []

for s in shipments:
    o = s["origin"]
    d = s["destination"]
    w = s["week"]
    service_strings = get_service_strings(o, d)
    for c in carriers:
        # find the service string with the minimum transit time for the selected carrier
        transit_time_dict = transit_times.get((o, d, w), {})
        min_transit_time = min([transit_time_dict.get((c, s), float('inf')) for s in service_strings])
        min_service_string = [s for s in service_strings if transit_time_dict.get((c, s), float('inf')) == min_transit_time][0]
        if x[XKey(s["order_number"], o, d, c, w)].value() == 1:
            pending_shipments.append({"order_number": s["order_number"], "origin": o, "destination": d, "carrier": c, "service_string": min_service_string, "week": w})
            # append any additional information you want to include
            

In [39]:
df_pending_shipments = pd.DataFrame(pending_shipments)
print(df_pending_shipments)

   order_number   origin   destination   carrier   service_string   week
0             2  origin2  destination2  carrier1  service_string4  week2


In [26]:
df_pending_shipments = pd.DataFrame(pending_shipments)
print(df_pending_shipments)

   order_number   origin   destination   carrier       service_string   week
0             1  origin2  destination2  carrier1  some_service_string  week1
1             2  origin2  destination2  carrier1  some_service_string  week2
